### Models by other users posted on GitHub

For reference

In [ ]:
# dyelax model
model = Sequential([
        Conv2D(32, 3, 3, input_shape=(32, 16, 1), border_mode='same', activation='relu'),
        Conv2D(64, 3, 3, border_mode='same', activation='relu'),
        Dropout(0.5),
        Conv2D(128, 3, 3, border_mode='same', activation='relu'),
        Conv2D(256, 3, 3, border_mode='same', activation='relu'),
        Dropout(0.5),
        Flatten(),
        Dense(1024, activation='relu'),
        Dense(512, activation='relu'),
        Dense(128, activation='relu'),
        Dense(1, name='output', activation='tanh'),
    ])
model.compile(optimizer=Adam(lr=FLAGS.lrate), loss='mse')

In [ ]:
# pkern model
input_layer = Input(shape=get_cropped_shape((*IMG_SIZE, 3), CROPPING))
base_model = VGG16(weights='imagenet', include_top=False, input_tensor=input_layer)

# Remove the last block of the VGG16 net.
[base_model.layers.pop() for _ in range(4)]
base_model.outputs = [base_model.layers[-1].output]
base_model.layers[-1].outbound_nodes = []

# Make sure pre trained layers from the VGG net don't change while training.
for layer in base_model.layers:
    layer.trainable = False

# Add last block to the VGG model with modified sub sampling.
layer = base_model.outputs[0]
layer = Convolution2D(512, 3, 3, subsample=(2, 2), activation='relu', border_mode='same', name='block5_conv1')(
    layer)
layer = Convolution2D(512, 3, 3, subsample=(1, 2), activation='relu', border_mode='same', name='block5_conv2')(
    layer)
layer = Convolution2D(512, 3, 3, subsample=(1, 2), activation='relu', border_mode='same', name='block5_conv3')(
    layer)

layer = Flatten()(layer)
layer = Dropout(.2)(layer)
layer = Dense(2048, activation='relu', name='fc1')(layer)
layer = Dropout(.2)(layer)
layer = Dense(1024, activation='relu', name='fc2')(layer)
layer = Dropout(.5)(layer)
layer = Dense(1, activation='linear', name='predictions')(layer)

return Model(input=base_model.input, output=layer)

In [ ]:
# wonjunee model

kernel_size = (3,3)

# Initiating the model
model = Sequential()

# Crop 50 pixels from the top of the image and 20 from the bottom
model.add(Cropping2D(cropping=((50, 20), (0, 0)),
                     dim_ordering='tf', # default
                     input_shape=(160, 320, 3)))

# Resize the data
model.add(Lambda(resize))

model.add(Lambda(lambda x: x[:,:,:,0:1]))

model.add(Lambda(lambda x: (x/127.5) - 1.))

# Starting with the convolutional layer
# The first layer will turn 1 channel into 16 channels
model.add(Convolution2D(16, kernel_size[0], kernel_size[1],
                        border_mode='valid'))
model.add(Activation('relu'))

model.add(Convolution2D(8, kernel_size[0], kernel_size[1]))
model.add(Activation('relu'))

model.add(Convolution2D(4, kernel_size[0], kernel_size[1]))
model.add(Activation('relu'))

model.add(Convolution2D(2, kernel_size[0], kernel_size[1]))
model.add(Activation('relu'))

model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

# Flatten the matrix. The input has size of 360
model.add(Flatten())

model.add(Dense(16))
model.add(Activation('relu'))

model.add(Dense(16))
model.add(Activation('relu'))

model.add(Dense(16))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(1))

adam = Adam(lr=0.0001)
model.compile(optimizer=adam, loss="mse", metrics=['accuracy'])

model.summary()